In [1]:
import jupyter_black
from dotenv import load_dotenv
from garminconnect import Garmin
import os
import datetime
from typing import Any
import numpy as np
from pydantic import BaseModel, Field
from openai import OpenAI
from enum import Enum
import json

jupyter_black.load()
load_dotenv(".envrc")

True

In [2]:
GARMIN_EMAIL = os.getenv("GARMIN_EMAIL")
GARMIN_PASSWORD = os.getenv("GARMIN_PASSWORD")
GARMIN_TOKENSTORE = os.getenv("GARMIN_TOKENSTORE")

In [3]:
def start_garmin() -> Garmin:
    """Initialize Garmin connection."""
    try:
        GARMIN = Garmin()
        GARMIN.login(GARMIN_TOKENSTORE)
        print("Logged in with token store")
        return GARMIN
    except Exception as e:
        print(f"Could not login with token store: {e}")

    try:
        GARMIN = Garmin(
            email=GARMIN_EMAIL,
            password=GARMIN_PASSWORD,
            is_cn=False,
        )
        GARMIN.login()
        GARMIN.garth.dump(GARMIN_TOKENSTORE)
        return GARMIN
    except Exception as e:
        print(f"Could not login with email and password: {e}")
        raise

In [4]:
garmin = start_garmin()

Logged in with token store


In [5]:
def get_daily_health_summary(
    api: Any, start: datetime.date, end: datetime.date
) -> list[dict[str, Any]]:
    """
    One dict per day using get_user_summary() where possible.
    Fields: resting_heart_rate, exercise_minutes, stress_level, sleep_hours, sleep_score, steps, body_battery_final
    """

    def dstr(d: datetime.date) -> str:
        return d.isoformat()

    def daterange(a: datetime.date, b: datetime.date):
        for i in range((b - a).days + 1):
            yield a + datetime.timedelta(days=i)

    out: list[dict[str, Any]] = []

    for day in daterange(start, end):
        s = dstr(day)
        day_of_week = day.strftime("%A")
        summary = api.get_user_summary(s) or {}
        rhr = summary.get("restingHeartRate")
        steps = summary.get("totalSteps")
        stress_level = summary.get("averageStressLevel")
        body_battery_final = summary.get("bodyBatteryMostRecentValue") or summary.get(
            "mostRecentBodyBattery"
        )
        exercise_minutes = (summary.get("moderateIntensityMinutes") or 0) + (
            summary.get("vigorousIntensityMinutes") or 0
        )
        sleep_seconds = summary.get("sleepingSeconds")
        sleep_hours = round(sleep_seconds / 3600, 2)
        body_battery_start = summary.get("bodyBatteryAtWakeTime")
        total_distance_meters = summary.get("totalDistanceMeters")

        out.append(
            {
                "date": s,
                "day_of_week": day_of_week,
                "resting_heart_rate": rhr,
                "exercise_minutes": exercise_minutes,
                "stress_level": stress_level,
                "sleep_hours": sleep_hours,
                "steps": steps,
                "total_distance_meters": total_distance_meters,
                "body_battery_start_day": body_battery_start,
                "body_battery_end_day": body_battery_final,
            }
        )

    return out

In [6]:
today = datetime.date.today()
summary_for_today = get_daily_health_summary(garmin, today, today)
summary_for_today

[{'date': '2025-08-11',
  'day_of_week': 'Monday',
  'resting_heart_rate': 43,
  'exercise_minutes': 0,
  'stress_level': 22,
  'sleep_hours': 7.26,
  'steps': 2320,
  'total_distance_meters': 1963,
  'body_battery_start_day': 76,
  'body_battery_end_day': 48}]

In [7]:
past_7_days_start = today - datetime.timedelta(days=7)
past_7_days_end = today - datetime.timedelta(days=1)
summary_for_past_7_days = get_daily_health_summary(
    garmin, past_7_days_start, past_7_days_end
)
summary_for_past_7_days

[{'date': '2025-08-04',
  'day_of_week': 'Monday',
  'resting_heart_rate': 39,
  'exercise_minutes': 0,
  'stress_level': 20,
  'sleep_hours': 7.79,
  'steps': 2635,
  'total_distance_meters': 2243,
  'body_battery_start_day': 96,
  'body_battery_end_day': 29},
 {'date': '2025-08-05',
  'day_of_week': 'Tuesday',
  'resting_heart_rate': 39,
  'exercise_minutes': 53,
  'stress_level': 20,
  'sleep_hours': 6.4,
  'steps': 9694,
  'total_distance_meters': 10748,
  'body_battery_start_day': 82,
  'body_battery_end_day': 24},
 {'date': '2025-08-06',
  'day_of_week': 'Wednesday',
  'resting_heart_rate': 40,
  'exercise_minutes': 0,
  'stress_level': 27,
  'sleep_hours': 6.2,
  'steps': 10952,
  'total_distance_meters': 9311,
  'body_battery_start_day': 66,
  'body_battery_end_day': 15},
 {'date': '2025-08-07',
  'day_of_week': 'Thursday',
  'resting_heart_rate': 39,
  'exercise_minutes': 48,
  'stress_level': 28,
  'sleep_hours': 6.06,
  'steps': 9954,
  'total_distance_meters': 10540,
  'bod

In [8]:
def detect_trend(values, pct_threshold=5):
    recent, earlier = np.mean(values[-3:]), np.mean(values[:3])
    return (
        "up"
        if recent > earlier * (1 + pct_threshold / 100)
        else ("down" if recent < earlier * (1 - pct_threshold / 100) else "flat")
    )


def build_llm_context_md(
    summary_for_today: list[dict], summary_for_past_7_days: list[dict]
) -> str:
    """
    Creates a Markdown-formatted context string for an LLM.
    Clarifies:
    - The "7-day average" excludes today's value
    - The trend is based on the 7 days before today
    - Percent change is relative to that 7-day average
    """
    assert len(summary_for_today) == 1, "Expected 1 day of summary"
    today = summary_for_today[0]

    metrics = [key for key in today.keys() if key not in ["date", "day_of_week"]]
    lines = [
        f"# Daily Metrics Summary for {today['date']} ({today['day_of_week']})",
        "_Note: All comparisons use the **previous 7 days only**, excluding today._",
        "",
    ]

    better_is_lower = [
        "resting_heart_rate",
        "stress_level",
    ]

    for metric in metrics:
        today_val = today[metric]
        past_vals = [day[metric] for day in summary_for_past_7_days]
        avg_7d = sum(past_vals) / len(past_vals) if past_vals else 0
        delta_pct = ((today_val - avg_7d) / avg_7d * 100) if avg_7d else 0
        trend_dir = detect_trend(past_vals)
        arrow = "↑" if trend_dir == "up" else ("↓" if trend_dir == "down" else "→")

        lines.append(
            f"## {metric.replace('_', ' ').title()}\n"
            f"- Today's value ({today['date']}): {today_val}\n"
            f"- 7-day baseline average (excluding today): {avg_7d:.2f}\n"
            f"- Percent change vs. baseline: {delta_pct:+.1f}%\n"
            f"- Trend over previous 7 days: {trend_dir} {arrow}\n"
            f"- Better is lower: {metric in better_is_lower}\n"
        )

    return "\n".join(lines)


prompt = build_llm_context_md(summary_for_today, summary_for_past_7_days)

print(prompt)

# Daily Metrics Summary for 2025-08-11 (Monday)
_Note: All comparisons use the **previous 7 days only**, excluding today._

## Resting Heart Rate
- Today's value (2025-08-11): 43
- 7-day baseline average (excluding today): 40.29
- Percent change vs. baseline: +6.7%
- Trend over previous 7 days: up ↑
- Better is lower: True

## Exercise Minutes
- Today's value (2025-08-11): 0
- 7-day baseline average (excluding today): 44.57
- Percent change vs. baseline: -100.0%
- Trend over previous 7 days: up ↑
- Better is lower: False

## Stress Level
- Today's value (2025-08-11): 22
- 7-day baseline average (excluding today): 25.57
- Percent change vs. baseline: -14.0%
- Trend over previous 7 days: up ↑
- Better is lower: True

## Sleep Hours
- Today's value (2025-08-11): 7.26
- 7-day baseline average (excluding today): 7.00
- Percent change vs. baseline: +3.7%
- Trend over previous 7 days: up ↑
- Better is lower: False

## Steps
- Today's value (2025-08-11): 2320
- 7-day baseline average (excludin

In [ ]:
class DayType(str, Enum):
    TRAINING = "training"
    ACTIVE_RECOVERY = "active_recovery"
    REST = "rest"
    HIGH_STRESS = "high_stress"
    BALANCED = "balanced"


class DailySummary(BaseModel):
    day_type: DayType = Field(
        ...,
        description="Classification of the day based on activity and recovery metrics",
    )

    title: str = Field(..., description="One sentence summary of the day")
    emoji: str = Field(..., description="Emoji to represent the day type")
    observation: str = Field(
        ...,
        description="Two sentence observation about key metrics and patterns",
    )
    recommendation: str = Field(
        ...,
        description="Two sentence actionable recommendation for tomorrow",
    )

In [10]:
training_day_example = DailySummary(
    day_type=DayType.TRAINING,
    title="Strong training day with elevated activity across all metrics.",
    emoji="💪",
    observation="Exercise minutes doubled your baseline with 95 minutes of activity, supported by 18,500 steps. Despite the high training load, body battery started at a solid 85, indicating good recovery from yesterday.",
    recommendation="Consider an active recovery or rest day tomorrow to allow adaptation from today's effort. Prioritize sleep tonight to maintain your body battery levels and support muscle recovery.",
)

# Example 2: High Stress Day
high_stress_day_example = DailySummary(
    day_type=DayType.HIGH_STRESS,
    emoji="😫",
    title="Elevated stress and poor recovery despite minimal physical activity.",
    observation="Stress levels jumped 46% above baseline while sleep dropped to just 6.1 hours, resulting in a low body battery start of 45. Exercise and movement were minimal, suggesting stress is from non-physical sources.",
    recommendation="Focus on stress management techniques and aim for 8+ hours of sleep tonight. Consider light exercise like walking or yoga tomorrow, as gentle movement can help regulate stress levels.",
)
examples = [training_day_example, high_stress_day_example]
examples_str = "\n\n".join([str(example) for example in examples])

In [11]:
system_prompt = f"""
You will be given a summary of the user's health and fitness data for today, in comparison to the past 7 days.
Your goal is to generate a summary that will be shown in the user's smart watch. 
Your summary includes a type of day, a title, some observations and recommendations for the user. 
Your summary should be in JSON format.

EXAMPLE JSON OUTPUTS:
{examples_str}
"""

In [12]:
def llm(
    messages: list[dict], model: str, response_format: dict | None = None
) -> tuple[dict, str | None]:
    client = OpenAI(
        api_key=os.environ["DEEPSEEK_API_KEY"],
        base_url="https://api.deepseek.com",
    )
    response = client.chat.completions.create(
        model=model, messages=messages, response_format=response_format, temperature=0.0
    )
    message = response.choices[0].message

    if hasattr(message, "reasoning_content"):
        reasoning_content = message.reasoning_content
    else:
        reasoning_content = None

    return json.loads(message.content), reasoning_content

In [13]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]


response, reasoning = llm(messages, "deepseek-chat", {"type": "json_object"})
health_summary = DailySummary.model_validate(response)
print(health_summary)
print(f"Reasoning:\n{reasoning}")

day_type=<DayType.REST: 'rest'> title='Restful day with minimal activity and improved stress levels.' emoji='😌' observation='Today was marked by significantly reduced activity with no exercise minutes and steps well below the baseline. However, stress levels were lower than usual, and sleep duration slightly increased. The body battery ended the day much higher than the baseline, suggesting good energy conservation.' recommendation="Consider maintaining this restful pace if recovery is your goal. If you're planning to return to activity tomorrow, start with light exercises to gradually increase your heart rate and step count without overexerting."
Reasoning:
None


In [14]:
response = llm(messages, "deepseek-reasoner", {"type": "json_object"})
health_summary = DailySummary.model_validate(response)
print(health_summary)
print(f"Reasoning:\n{reasoning}")

KeyboardInterrupt: 